In [1]:
import os
import sys
sys.path.append('/home/jovyan/work')
# Exibir o caminho do diretório atual
print("Caminho do diretório atual (logado):", os.getcwd())
print("Python executable:", sys.executable)

Caminho do diretório atual: /home/jovyan/work
Python executable: /home/jovyan/.cache/pypoetry/virtualenvs/brazilian-e-commerce-project-WFljCIXp-py3.11/bin/python


In [ ]:
from dotenv import load_dotenv 
# Carregar variáveis de ambiente do arquivo .env
load_dotenv(os.path.join('config', '.env'))

In [ ]:
import psycopg2
from psycopg2 import sql
import pandas as pd
from sqlalchemy import create_engine
from pandas import DataFrame

# Configuração da conexão com o banco de dados
# Parâmetros de conexão
host = os.getenv('DB_HOST') # Nome do serviço no Docker
user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
port = os.getenv('DB_PORT')
dbname = os.getenv('DB_NAME')
new_dbname = os.getenv('NEW_DBNAME')  # Nome do novo banco de dados

## Prepara Banco SQL

Este notebook tem o objetivo de criar um banco de dados dentro do postgre_db para inserir os dados das bases CSV presentes na pasta `data\archive`.

In [3]:
def create_database(
        host: str, 
        user: str, 
        password: str, 
        port: str, 
        dbname: str, 
        new_dbname: str) -> None:
    """
    Cria um novo banco de dados.

    Args:
        host (str): Nome do serviço no Docker.
        user (str): Nome do usuário do banco de dados.
        password (str): Senha do banco de dados.
        port (str): Porta de conexão ao banco de dados.
        dbname (str): Nome do banco de dados existente ao qual conectar.
        new_dbname (str): Nome do novo banco de dados a ser criado.

    Returns:
        None
    """
    conn = None  # Inicializa conn como None
    try:
        # Conectar ao banco de dados existente
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        conn.autocommit = True

        # Criar um cursor
        cursor = conn.cursor()

        # Criar o novo banco de dados
        cursor.execute(sql.SQL("CREATE DATABASE {}").format(
            sql.Identifier(new_dbname))
        )

        print(f"Banco de dados '{new_dbname}' criado com sucesso!")

    except Exception as e:
        print(f"Erro ao criar o banco de dados: {e}")

    finally:
        # Fechar a conexão
        if conn:
            cursor.close()
            conn.close()

# Criar o novo banco de dados
create_database(host, user, password, port, dbname, new_dbname)


Erro ao criar o banco de dados: database "brasilian_e_commerce" already exists



In [4]:
def salvar_dataframe_postgresql(
    df: DataFrame, 
    table_name: str, 
    host: str = host, 
    user: str = user, 
    password: str= password, 
    port: str = port, 
    dbname: str = new_dbname
) -> None:
    """
    Salva um DataFrame Pandas em uma tabela do banco de dados PostgreSQL.

    Parâmetros:
    df (DataFrame): DataFrame a ser salvo.
    table_name (str): Nome da tabela no PostgreSQL.
    user (str): Usuário do PostgreSQL.
    password (str): Senha do PostgreSQL.
    host (str): Host onde o PostgreSQL está rodando (ex: 'localhost').
    port (str): Porta do PostgreSQL (ex: '5432').
    dbname (str): Nome do banco de dados PostgreSQL.

    Retorno:
    None
    """
    try:
        # Criando o engine de conexão
        engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')
        
        # Salvando o DataFrame no PostgreSQL
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"Dados salvos com sucesso na tabela '{table_name}' no PostgreSQL!")
    
    except Exception as e:
        print(f"Erro ao salvar os dados: {e}")


In [5]:
olist_customers = pd.read_csv("/home/jovyan/work/data/archive/olist_customers_dataset.csv")
olist_order_items = pd.read_csv("/home/jovyan/work/data/archive/olist_order_items_dataset.csv")
olist_order_payments = pd.read_csv("/home/jovyan/work/data/archive/olist_order_payments_dataset.csv")
olist_order_reviews = pd.read_csv("/home/jovyan/work/data/archive/olist_order_reviews_dataset.csv")
olist_orders = pd.read_csv("/home/jovyan/work/data/archive/olist_orders_dataset.csv")
olist_products = pd.read_csv("/home/jovyan/work/data/archive/olist_products_dataset.csv")
olist_sellers = pd.read_csv("/home/jovyan/work/data/archive/olist_sellers_dataset.csv")
pdt_cat_name_trans = pd.read_csv("/home/jovyan/work/data/archive/product_category_name_translation.csv")

In [6]:
salvar_dataframe_postgresql(olist_customers, 'olist_customers')

Dados salvos com sucesso na tabela 'olist_customers' no PostgreSQL!


In [7]:
salvar_dataframe_postgresql(olist_order_items, 'olist_order_items')

Dados salvos com sucesso na tabela 'olist_order_items' no PostgreSQL!


In [8]:
salvar_dataframe_postgresql(olist_order_payments, 'olist_order_payments')

Dados salvos com sucesso na tabela 'olist_order_payments' no PostgreSQL!


In [9]:
salvar_dataframe_postgresql(olist_order_reviews, 'olist_order_reviews')

Dados salvos com sucesso na tabela 'olist_order_reviews' no PostgreSQL!


In [10]:
salvar_dataframe_postgresql(olist_orders, 'olist_orders')

Dados salvos com sucesso na tabela 'olist_orders' no PostgreSQL!


In [11]:
salvar_dataframe_postgresql(olist_products, 'olist_products')

Dados salvos com sucesso na tabela 'olist_products' no PostgreSQL!


In [12]:
salvar_dataframe_postgresql(olist_sellers, 'olist_sellers')

Dados salvos com sucesso na tabela 'olist_sellers' no PostgreSQL!


In [13]:
salvar_dataframe_postgresql(pdt_cat_name_trans, 'pdt_cat_name_trans')

Dados salvos com sucesso na tabela 'pdt_cat_name_trans' no PostgreSQL!
